In [8]:
%pwd

'/home/ljw/Documents'

In [2]:
ll

total 6076
-rw------- 1 root 5218936 Jun 28 11:59  IPCC_AR6_SYR_LongerReport.pdf
-rw-r--r-- 1 root   38223 Jul  1 10:08 'llm_gen_TODO_local invoke Qwen using Transformer and ollama.ipynb'
-rw-r--r-- 1 root    2544 Jul  1 11:46  llm.py
-rw-r--r-- 1 root  179093 Jun 27 12:42  lm-eval.ipynb
-rw-r--r-- 1 root  350225 Jun 28 11:54  opencompass.ipynb
-rw-r--r-- 1 root  419337 Jun 28 16:40  ragas.ipynb


In [3]:
!pip install ipywidgets 

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/d4/17/8b2ce5765dd423433d2e0727712629c46152fb0bc706b0977f847480f262/ipywidgets-8.1.3-py3-none-any.whl (139 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/93/c1/68423f43bc95d873d745bef8030ecf47cd67f932f20b3f7080a02cff43ca/widgetsnbextension-4.0.11-py3-none-any.whl (2.3 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/65/f6/659ca44182c86f57977e946047c339c717745fda9f43b7ac47f274e86553/jupyterlab_widgets-3.0.11-py3-none-any.whl (214 kB)


### LLM Invoke

In [5]:
!pip install langchain_huggingface

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/39/ce/ad7f50a6289cf562747df9a966b4d60a848571db2e57ad8d00dca2478096/langchain_huggingface-0.0.3-py3-none-any.whl (17 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/58/4b/922436953394e1bfda05e4bf1fe0e80f609770f256c59a9df7a9254f3e0d/sentence_transformers-3.0.1-py3-none-any.whl (227 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/40/4f/eb78de4af3b17b589f43a369cbf0c3a7173f25c3d2cd93068852c07689aa/tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 3.0 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/cb/e2/1bd899d3eb60c6495cf5d0d2885edacac08bde7a1407eadeb2ab36eca3c7/torch-2.3.1-cp310-cp310-manylinux1_x86_64.whl (779.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 3.2 MB/s eta 0:00:00m eta 0:00:0

  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/eb/d5/c68b1d2cdfcc59e72e8a5949a37ddb22ae6cade80cd4a57a84d4c8b55472/nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/7e/00/6b218edd739ecfc60524e585ba8e6b00554dd908de2c9c66c1af3e44e18d/nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/ff/74/a2e2be7fb83aaedec84f391f082cf765dfb635e7caa9b49065f73e4835d8/nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/37/6d/121efd7382d5b0284239f4ab1fc1590d86d34ed4a4a2fdb13b30ca8e5740/nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/86/94/eb540db023ce1d162e7bea9f8f5aa781d57c65aed513c33ee9a5123ead4d/nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached http

In [1]:
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_core.prompts import PromptTemplate

model_id = "/opt/loc-llm/Qwen1.5-4B"

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)
gpu_llm = HuggingFacePipeline.from_model_id(
    model_id=model_id,
    task="text-generation",
    device=0,  # replace with device_map="auto" to use the accelerate library.
    pipeline_kwargs={"max_new_tokens": 100},
)

gpu_chain = prompt | gpu_llm.bind(skip_prompt=True)

question = "What is electroencephalography?"

print(gpu_chain.invoke("hello"))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Question: hello

Answer: Let's think step by step. The question is asking for a greeting. A common greeting is "hello". So the answer is hello.


### Transformer -> OutOfMemoryError: CUDA out of memory

In [1]:
# %load llm.py
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_core.prompts import PromptTemplate

model_id = "/opt/loc-llm/Qwen1.5-7B-Chat/"

template = """
请使用以下内容作为基础生成中文的Excel问答对，并确保做到以下要求：
	- Excel表格形式返回：将生成的问答对以标准的 Excel 表格呈现。

Excel表格要求:
1. 确保包含至少以下列:
	- 问题编号（QID）: 用于标识每个问题的独特性；
    - 问题内容（Question）: 每个具体的问题表述；
    - 答案描述（Answer）: 对应于每个问题的详细且准确解答。
    
{context}
请确保您的回答遵循这些指示，并在最终结果中体现。

"""

prompt = PromptTemplate.from_template(template)
gpu_llm = HuggingFacePipeline.from_model_id(
    model_id=model_id,
    task="text-generation",
    device=0,  # replace with device_map="auto" to use the accelerate library.
    pipeline_kwargs={"max_new_tokens": 20},
)


print(gpu_llm.invoke("请生成一条中文的问答对，问题随意，但用excel格式表示"))

gpu_chain = prompt | gpu_llm.bind(skip_prompt=True)

context = """
在人工智能领域，由于大模型（LLM）技术的发展以及其广阔的市场前景，MaaS 以及开源大模型呈现出百家争鸣的景象。现阶段，大型语言模型的开发和应用已经成为各个领域智能化提升的重要方向。为了利用大模型实现业务和产品的提升或创新，就需要对大模型进行系统的评估。
LLM评估是指在人工智能系统中评估和改进语言和语言模型的过程。在人工智能领域，特别是在自然语言处理（NLP）及相关领域，LLM评估具有至高无上的地位。通过评估语言生成和理解模型，LLM评估有助于细化人工智能驱动的语言相关任务和应用程序，确保在语言发挥关键作用的各种场景中增强准确性和适应性。
随着大模型的版本升级和应用的持续，对大模型的评估也绝非一次性，而是需要多次迭代的过程。建立一个有效的、可持续的评估过程非常重要。如今，许多大模型服务通过LLMOps实现了CI、CE、CD（持续集成、持续评估、持续部署），大大提高了大模型的可用性。但在大模型微调、开源大模型应用、基于RAG的架构以及行业应用等层面，我们仍然需要了解大模型的评测技术。

"""

print(gpu_chain.invoke({"context": context}))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 

### ollama

In [1]:
!ollama list

NAME          	ID          	SIZE  	MODIFIED    
mistral:latest	2ae6f6dd7a3d	4.1 GB	10 days ago	
llama2:latest 	78e26419b446	3.8 GB	10 days ago	
phi3:3.8b     	64c1188f2485	2.4 GB	13 days ago	
phi3:14b      	1e67dff39209	7.9 GB	13 days ago	
phi3:latest   	64c1188f2485	2.4 GB	13 days ago	
llama3:8b     	365c0bd3c000	4.7 GB	13 days ago	
qwen:72b      	e1c64582de5c	41 GB 	13 days ago	
qwen2:latest  	e0d4e1163c58	4.4 GB	13 days ago	


In [3]:
# LangChain supports many other chat models. Here, we're using Ollama
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import pandas as pd

llm = ChatOllama(model="qwen2:latest")

template = """
请使用以下内容作为基础生成中文的CSV问答对，并确保做到以下要求：
	- CSV形式返回：将生成的问答对以标准的CSV呈现。

CSV要求:
1. 确保包含至少以下列:
	- 问题编号（QID）: 用于标识每个问题的独特性；
    - 问题内容（Question）: 每个具体的问题表述；
    - 答案描述（Answer）: 对应于每个问题的详细且准确解答。

CSV格式举例说明：
	```
	"问题编号","问题内容","答案描述"
	"Q1","什么是LLM","LLM是指大模型，也是AI"
	```
    
{context}
请确保您的回答遵循这些指示，并在最终结果中体现。

"""

prompt = ChatPromptTemplate.from_template(template)
chain = prompt | llm | StrOutputParser()

context = """
在人工智能领域，由于大模型（LLM）技术的发展以及其广阔的市场前景，MaaS 以及开源大模型呈现出百家争鸣的景象。现阶段，大型语言模型的开发和应用已经成为各个领域智能化提升的重要方向。为了利用大模型实现业务和产品的提升或创新，就需要对大模型进行系统的评估。
LLM评估是指在人工智能系统中评估和改进语言和语言模型的过程。在人工智能领域，特别是在自然语言处理（NLP）及相关领域，LLM评估具有至高无上的地位。通过评估语言生成和理解模型，LLM评估有助于细化人工智能驱动的语言相关任务和应用程序，确保在语言发挥关键作用的各种场景中增强准确性和适应性。
随着大模型的版本升级和应用的持续，对大模型的评估也绝非一次性，而是需要多次迭代的过程。建立一个有效的、可持续的评估过程非常重要。如今，许多大模型服务通过LLMOps实现了CI、CE、CD（持续集成、持续评估、持续部署），大大提高了大模型的可用性。但在大模型微调、开源大模型应用、基于RAG的架构以及行业应用等层面，我们仍然需要了解大模型的评测技术。

"""

csv_str = chain.invoke({"context": context})
print(csv_str)
from io import StringIO
df = pd.read_csv(StringIO(csv_str))

# 查看DataFrame
df

```
"问题编号","问题内容","答案描述"
"Q1","LLM评估是什么？","LLM评估是在人工智能系统中评估和改进语言和语言模型的过程，对自然语言处理（NLP）及相关领域至关重要。它帮助细化AI驱动的语言相关任务和应用，确保各种场景中的准确性与适应性。"
"Q2","大模型的评估为何需要多次迭代？","随着大模型的版本升级和应用持续发展，评估过程需多次迭代以确保其性能优化并保持竞争力。建立有效的评估流程对于提升大模型可用性至关重要。"
"Q3", "LLMOps是什么？","LLMOps是利用持续集成（CI）、持续评估（CE）和持续部署（CD）来管理大型语言模型（LLM）的一种实践方法，旨在提高大模型的服务质量和效率。"
"Q4", "哪些方面需要了解关于大模型的评测技术？","在大模型微调、开源大模型应用、基于RAG的架构以及行业应用层面，都需要了解相应的评测技术以优化模型性能和实际应用场景匹配度。"
```


,,```
问题编号,问题内容,答案描述
Q1,LLM评估是什么？,LLM评估是在人工智能系统中评估和改进语言和语言模型的过程，对自然语言处理（NLP）及相关领...
Q2,大模型的评估为何需要多次迭代？,随着大模型的版本升级和应用持续发展，评估过程需多次迭代以确保其性能优化并保持竞争力。建立有效...
Q3,"""LLMOps是什么？""",LLMOps是利用持续集成（CI）、持续评估（CE）和持续部署（CD）来管理大型语言模型（L...
Q4,"""哪些方面需要了解关于大模型的评测技术？""",在大模型微调、开源大模型应用、基于RAG的架构以及行业应用层面，都需要了解相应的评测技术以优...
```,NaN,NaN


In [5]:
!pip install PyPDF2

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.7 MB/s eta 0:00:001.2 MB/s eta 0:00:01


> 多次执行其结果以及基于context生成的问题未必正确，还是需要从中筛选，也从中可以看出要根据语义数据才能生成有效的问答对

In [10]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

import PyPDF2
import pandas as pd
import os

# 读取PDF并提取文本
file_path = './IPCC_AR6_SYR_LongerReport.pdf'
llm = ChatOllama(model="qwen2:latest")
template = """
请使用以下内容作为基础生成中文的CSV问答对，并确保做到以下要求：
	- CSV格式返回：将生成的一道问答对以标准的CSV格式直接呈现。

## CSV格式要求:
1. 确保包含至少以下列:
	- 问题编号（QID）: 用于标识每个问题的独特性；
    - 问题内容（Question）: 每个具体的问题表述；
    - 问题来源（Context）: 抽取你觉得有用的可以作为参考生成问答对的PDF的`context的内容`
    - 答案描述（Answer）: 对应于每个问题的详细且准确解答。
2. 以**逗号**相隔

## CSV格式举例说明：
	```
	"问题编号","问题内容","问题来源","答案描述"
	"Q1","哪些因素影响了全球平均表面气温的变化","The colour coding indicates the assessed conﬁdence in / likelihood76 of the observed change and the human contribution as a driver or main driver (speciﬁed in that case) ","全球平均表面气温变化受多种因素影响，包括但不限于气候变化系统组件的观测变化及对人类影响的归因。"
	"Q2","在世界不同地区观察到的变化类型与对人类贡献的认识有什么关系？","Section 2, IPCC AR6 WGI参考区域: 北美(NWN、NEN等), 中美洲(NCA), 南美洲(SAM、WAF等), 欧洲(GIC、NEU等), 非洲(MED、SAH等), 亚洲(RAR、ARP等)","在世界不同地区观察到的变化类型与对人类贡献的认识关系密切，高、中和低程度的数据/文献支持表明了气候变化的程度及人类活动的影响。"
	```

## context的内容：
```    
{context}
```

请确保您的回答遵循这些指示，并在最终结果中体现。**请确保返回符合以逗号相隔的CSV格式的多条问答对，不需要其他多余的内容！**

"""

prompt = ChatPromptTemplate.from_template(template)
chain = prompt | llm | StrOutputParser()

pdf_file_obj = open(file_path, 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file_obj)

page_num = 13
page_obj = pdf_reader.pages[page_num]
context = page_obj.extract_text()

csv_str = chain.invoke({"context": context})
from io import StringIO
df = pd.read_csv(StringIO(csv_str))
print(csv_str)
print()
df 
        
# for page_num in range(len(pdf_reader.pages)):
#     if page_num > 13:
#         pdf_file_obj.close()
#         break

"问题编号","问题内容","问题来源","答案描述"
"Q1","气候变化如何影响人类和自然系统？","第2节, IPCC AR6 WGI","气候变化已经影响了全球范围内的各种人类和自然系统。在受影响最大的地区，尽管人们贡献较小的气候变暖，但这些区域也最容易受到影响。"
"Q2","世界上不同地区的变化类型与人类贡献的认识有何关系？","第1节, 第2节, IPCC AR6 WGI参考区域: 北美(NWN、NEN等), 中美洲(NCA), 南美洲(SAM、WAF等), 欧洲(GIC、NEU等), 非洲(MED、SAH等), 亚洲(RAR、ARP等)","世界不同地区的变化类型与人类贡献的认识密切相关。高、中和低数据/文献支持表明了气候变化程度及人类活动影响。"



,问题编号,问题内容,问题来源,答案描述
0,Q1,气候变化如何影响人类和自然系统？,"第2节, IPCC AR6 WGI",气候变化已经影响了全球范围内的各种人类和自然系统。在受影响最大的地区，尽管人们贡献较小的气候...
1,Q2,世界上不同地区的变化类型与人类贡献的认识有何关系？,"第1节, 第2节, IPCC AR6 WGI参考区域: 北美(NWN、NEN等), 中美洲(...",世界不同地区的变化类型与人类贡献的认识密切相关。高、中和低数据/文献支持表明了气候变化程度及...
